#Session 1: Application of Inverse Problem and Data Assimilation

## 🤖 AI Coding Policy

**Core Philosophy:** Treat AI as your **Co-pilot**, not your **Autopilot**.

✅ **Encouraged:**
* Using AI to explain syntax, complex libraries, or error messages (Debugging).
* Generating boilerplate code or optimizing specific snippets.
* Brainstorming different implementation approaches.

❌ **Prohibited:**
* **Copy-pasting homework prompts directly into AI to generate solutions.**
* Submitting code that you do not understand or cannot explain.

⚠️ **Be Careful:**
1.  **Hallucinations:** AI makes confident mistakes. You must verify correctness.
2.  **Accountability:** You are fully responsible for every line of code you submit.

## 1. Problem Setup: Bayesian Inverse Problems

We consider the general inverse problem of finding an unknown parameter $u$ from noisy data (Observation) $y$:

$$y = G(u) + \eta.$$

### Bayesian Formulation

* **Prior:** **$u\sim\rho(u)$**.
* **Likelihood:** $y|u\sim\nu(y - G(u))$.
* **Posterior:**
$$\pi^y(u) \propto\mathbb{P}(u)\mathbb{P}(y|u)= \nu(y - G(u)) \cdot \rho(u)$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import ipywidgets as widgets

plt.style.use('seaborn-v0_8-whitegrid')

# --- Helper Functions ---

def G_identity(u):
    """
    Function: Identity map.
    Input: u (array or float)
    Output: u (array or float)
    """
    return u

def G_square(u):
    """
    Function: Square map.
    Input: u (array or float)
    Output: u^2 (array or float)
    """
    return u**2

G_MAPS = {
    'Linear: G(u) = u': G_identity,
    'Non-linear: G(u) = u^2': G_square
}

def compute_densities(u_grid, y_obs, sigma_prior, sigma_noise, G_func):
    """
    Function: Computes Prior, Likelihood, and Posterior densities.
    Input:
        u_grid (np.ndarray): Domain
        y_obs (float): Observation
        sigma_prior, sigma_noise (float): Standard deviations
        G_func (callable): Forward map
    Output: tuple (rho, likelihood, pi_y)
    """
    rho = norm.pdf(u_grid, loc=0, scale=sigma_prior)
    likelihood = norm.pdf(y_obs, loc=G_func(u_grid), scale=sigma_noise)
    pi_y = rho * likelihood
    return rho, likelihood, pi_y

def viz_interactive(map_name, y_obs=4.0, gamma=1.0):
    """
    Function: Plots Prior, Likelihood, and Posterior on a single figure.
    Input: map_name (str), y_obs (float), gamma (float)
    Output: None (displays plot)
    """
    # Parameters
    sigma_prior = 2.0
    u_grid = np.linspace(-6, 6, 1000)
    G_func = G_MAPS[map_name]

    # Compute densities
    rho, likelihood, pi_y = compute_densities(u_grid, y_obs, sigma_prior, gamma, G_func)

    # Normalize posterior

    normalization = np.trapezoid(pi_y, u_grid)
    pi_y_norm = pi_y / normalization if normalization > 0 else pi_y

    # Plotting
    fig, ax = plt.subplots(figsize=(10, 6))

    # 1. Prior
    ax.plot(u_grid, rho, 'b--', lw=1.5, label=r'Prior $\rho(u)$')
    ax.fill_between(u_grid, rho, color='blue', alpha=0.05)

    # 2. Likelihood
    ax.plot(u_grid, likelihood, 'r:', lw=2, label=r'Likelihood $L(u)$')

    # 3. Posterior
    ax.plot(u_grid, pi_y_norm, 'k-', lw=2.5, label=r'Posterior $\pi^y(u)$')
    ax.fill_between(u_grid, pi_y_norm, color='black', alpha=0.1)

    # Mark roots if applicable (visual guide for modes)
    if 'u^2' in map_name and y_obs > 0:
        root = np.sqrt(y_obs)
        ax.axvline(root, color='green', linestyle='-', alpha=0.3)
        ax.axvline(-root, color='green', linestyle='-', alpha=0.3, label='Truth Roots')

    ax.set_title(fr'Bayesian Inference (y={y_obs}, $\gamma$={gamma})', fontsize=14)
    ax.set_xlabel('$u$')
    ax.set_ylabel('Density')
    ax.legend(loc='upper right', frameon=True)

    plt.tight_layout()
    plt.show()

# --- Widget Setup ---
# Note: continuous_update=False prevents updates while dragging
print("Select Forward Map G(u) and adjust parameters:")
widgets.interactive(
    viz_interactive,
    map_name=widgets.Dropdown(options=list(G_MAPS.keys()), value='Non-linear: G(u) = u^2', description='Map:'),
    y_obs=widgets.FloatSlider(value=4.0, min=-5.0, max=10.0, step=0.1, description=r'$y$ (Obs):', continuous_update=False),
    gamma=widgets.FloatSlider(value=1.0, min=0.1, max=4.0, step=0.1, description=r'$\gamma$ (Noise):', continuous_update=False)
)

## 2. Linear Inverse Problems: Denoising & Deblurring

We generalize our problem to high-dimensional linear inverse problems:

$$y = G(u) + \eta$$

where $u$ is the image we want to recover, and $G$ is a linear operator. To solve this, we minimize the following objective function (Variational Approach):

$$u_{MAP} = \operatorname*{argmin}_u \left( \underbrace{\frac{1}{2}\|G(u) - y\|^2}_{\text{Data Fidelity}} + \underbrace{\lambda \|\nabla u\|_1}_{\text{Regularization}} \right)$$

### Bayesian Interpretation
Why this specific minimization? It corresponds exactly to finding the **Maximum A Posteriori (MAP)** estimate in a Bayesian framework: $u_{MAP} = \operatorname*{argmax}_u \pi^y(u)$.

Taking the negative logarithm of the posterior, maximization becomes minimization:
$$-\log \pi^y(u) = -\log \text{Likelihood} - \log \text{Prior} + C$$

1.  **Likelihood ($\text{Data Fidelity}$):**
    Assuming the noise $\eta$ is Gaussian $\mathcal{N}(0, I)$, the likelihood of observing $y$ given $u$ is:
    $$\nu(y|u) \propto \exp\left(-\frac{1}{2}\|G(u) - y\|^2\right)$$

2.  **Prior ($\text{Regularization}$):**
    The Total Variation (TV) regularization implies a specific prior belief about the image statistics:
    $$\rho(u) \propto \exp\left(-\lambda \|\nabla u\|_1\right)$$
    This is a **Laplace distribution** on the image gradients. Unlike a Gaussian prior (which smooths everything), a Laplace prior allows for "sparse" gradients—meaning it permits sharp edges (large jumps) while keeping flat regions smooth.

### The Role of $G$
* **Denoising:** $G = I$ (Identity). The data is simply a noisy version of $u$.
* **Deblurring (Deconvolution):** $G$ is a convolution operator (e.g., Gaussian Blur). The data is both blurry and noisy, making the problem significantly harder (more ill-posed).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import data, img_as_float
from skimage.restoration import denoise_tv_chambolle
from skimage.util import random_noise
from scipy.ndimage import gaussian_filter
import ipywidgets as widgets
from IPython.display import display, clear_output

# Style setup
plt.style.use('seaborn-v0_8-whitegrid')

def text_image(size=128):
    # Manually draw block letters "CMS"
    img = np.zeros((size, size))

    # Letter C
    img[40:90, 10:20] = 1.0   # Left vertical
    img[40:50, 10:40] = 1.0   # Top horizontal
    img[80:90, 10:40] = 1.0   # Bottom horizontal

    # Letter M (Blocky style)
    img[40:90, 50:60] = 1.0   # Left vertical
    img[40:90, 80:90] = 1.0   # Right vertical
    img[40:50, 60:80] = 1.0   # Top bridge
    img[50:70, 65:75] = 1.0   # Middle drop

    # Letter S
    img[40:50, 100:125] = 1.0 # Top
    img[60:70, 100:125] = 1.0 # Middle
    img[80:90, 100:125] = 1.0 # Bottom
    img[40:65, 100:110] = 1.0 # Top-Left connector
    img[65:90, 115:125] = 1.0 # Bottom-Right connector

    return img

# --- 1. Data Loading & Helpers ---
def load_image(name):
    if name == 'Cameraman':
        return img_as_float(data.camera())
    elif name == 'Text':
        img = text_image()
        return img

def G_blur(u, sigma=2.0):
    return gaussian_filter(u, sigma=sigma)

# --- 2. Solver Logic ---
def solve_inverse_problem(y, task_type, tv_weight, n_iters=50):
    u = np.copy(y)
    step_size = 0.9

    if task_type == 'Denoising (G=I)':
        return denoise_tv_chambolle(y, weight=tv_weight)

    elif task_type == 'Deblurring (G=Blur)':
        for _ in range(n_iters):
            residual = G_blur(u) - y
            gradient = G_blur(residual)
            u = u - step_size * gradient
            u = denoise_tv_chambolle(u, weight=tv_weight)
        return u

# --- 3. UI & Interaction Setup ---

# A. Define Widgets
w_img = widgets.Dropdown(
    options=['Cameraman', 'Text'],
    value='Cameraman',
    description='Image:'
)

w_task = widgets.Dropdown(
    options=['Denoising (G=I)', 'Deblurring (G=Blur)'],
    value='Denoising (G=I)',
    description='Task:'
)

w_noise = widgets.FloatSlider(
    value=0.1, min=0.0, max=0.5, step=0.05,
    description=r'Noise $\sigma$:'
)

w_tv = widgets.FloatSlider(
    value=0.1, min=0.01, max=0.5, step=0.01,
    description=r'TV $\lambda$:'
)

btn_run = widgets.Button(
    description='Run Reconstruction',
    button_style='success',
    icon='play'
)

out_plot = widgets.Output()

# B. Callback Function
def on_button_clicked(b):
    # Lock the button to prevent double clicks
    btn_run.disabled = True
    btn_run.description = "Running..."

    with out_plot:
        clear_output(wait=True)
        try:
            # 1. Get Params
            img_name = w_img.value
            task = w_task.value
            noise_level = w_noise.value
            tv_weight = w_tv.value

            # 2. Compute
            u_true = load_image(img_name)

            if task == 'Denoising (G=I)':
                y_clean = u_true
            else:
                y_clean = G_blur(u_true, sigma=2.0)

            y_obs = random_noise(y_clean, mode='gaussian', var=noise_level**2)
            u_est = solve_inverse_problem(y_obs, task, tv_weight)

            # 3. Plot
            fig, axes = plt.subplots(1, 3, figsize=(16, 5))

            axes[0].imshow(u_true, cmap='gray')
            axes[0].set_title(r"Ground Truth $u_{true}$", fontsize=14)
            axes[0].axis('off')

            axes[1].imshow(y_obs, cmap='gray')
            axes[1].set_title(fr"Observation $y$ ({task})", fontsize=14)
            axes[1].axis('off')

            axes[2].imshow(u_est, cmap='gray')
            axes[2].set_title(fr"Reconstruction ($\lambda={tv_weight}$)", fontsize=14)
            axes[2].axis('off')

            plt.tight_layout()

            # Use specific display function instead of plt.show()
            display(fig)
            plt.close(fig) # Prevent memory leak

        except Exception as e:
            print(f"Error occurred: {e}")
            import traceback
            traceback.print_exc()
        finally:
            # Reset button state
            btn_run.disabled = False
            btn_run.description = 'Run Reconstruction'

btn_run.on_click(on_button_clicked)

# C. Layout
ui = widgets.VBox([
    widgets.HBox([w_img, w_task]),
    widgets.HBox([w_noise, w_tv]),
    btn_run
])

print("Set parameters and click 'Run Reconstruction':")
display(ui, out_plot)

# Run once at startup to ensure something is visible
on_button_clicked(None)

## 3. Data-Driven Approach: Learning a Direct Mapping

Instead of solving an optimization problem for each new input, we train a neural network $\mathcal{F}_\theta$ to learn a direct, **deterministic mapping**:

$$u \approx \mathcal{F}_\theta(y)$$

**The Hidden Prior:**
Unlike the TV method where we explicitly wrote down the prior, here the prior is implicitly embedded in the network weights during the training process. By learning from massive datasets, the network encodes the statistical properties of natural images directly into its parameters.

We demonstrate this with two examples:
1.  **Super-Resolution (SR):** The network maps the low-diensional observation to a high-dimensional output.
2.  **Inpainting:** The network deterministically fills in missing regions.

In [ ]:
# --- 0. Environment Setup ---
try:
    import transformers
except ImportError:
    print("Installing HuggingFace Transformers...")
    !pip install -q git+https://github.com/huggingface/transformers.git

import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from skimage import data
from transformers import Swin2SRForImageSuperResolution, Swin2SRImageProcessor

# Check for GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# --- 1. Load Pre-trained Model (SwinIR) ---
model_name = "caidas/swin2SR-classical-sr-x4-64"
print(f"Loading model: {model_name} ...")

processor = Swin2SRImageProcessor.from_pretrained(model_name)
model = Swin2SRForImageSuperResolution.from_pretrained(model_name).to(device)
model.eval()
print("Model loaded successfully!")

# --- 2. Helper Functions ---

def prepare_data_pair(img_hr_pil, crop_size=256):
    """
    Crops the center of the image and simulates downsampling (degradation).
    Returns:
        img_hr: High-Res Ground Truth (cropped)
        img_lr: Low-Res Observation (downsampled)
    """
    w, h = img_hr_pil.size
    left = (w - crop_size)/2
    top = (h - crop_size)/2
    img_hr_cropped = img_hr_pil.crop((left, top, left+crop_size, top+crop_size))

    # Simulate degradation: Downsample by 4x
    scale_factor = 4
    w_lr, h_lr = img_hr_cropped.size[0] // scale_factor, img_hr_cropped.size[1] // scale_factor
    img_lr = img_hr_cropped.resize((w_lr, h_lr), resample=Image.BICUBIC)

    return img_hr_cropped, img_lr

def solve_inverse_problem(img_lr_pil):
    """
    Runs SwinIR inference to recover high-res image.
    """
    inputs = processor(img_lr_pil, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs)

    # Post-process: Tensor -> PIL Image
    output_tensor = outputs.reconstruction.data.squeeze().float().cpu().clamp_(0, 1)
    output_numpy = output_tensor.numpy().transpose(1, 2, 0)
    return Image.fromarray((output_numpy * 255.0).round().astype(np.uint8))

# --- 3. Execution Pipeline ---
print("\nProcessing Astronaut image...")

# Load standard Astronaut image
img_astro_full = Image.fromarray(data.astronaut())

# Prepare High-Res and Low-Res pair
hr_astro, lr_astro = prepare_data_pair(img_astro_full, crop_size=256)

# Method A: Deep Learning Inverse (SwinIR)
sr_astro_dl = solve_inverse_problem(lr_astro)

# Method B: Classical Inverse (Bicubic Baseline)
sr_astro_bicubic = lr_astro.resize(hr_astro.size, resample=Image.BICUBIC)

# --- 4. Visualization ---
print("Visualizing results...")
fig, axes = plt.subplots(1, 4, figsize=(20, 6))

cols = ["Ground Truth ($x$)", "Observation ($y$)\n(Low Res Input)", "Classical Inverse\n(Bicubic)", "Deep Learning Inverse\n(SwinIR)"]

# Plotting
axes[0].imshow(hr_astro)
axes[1].imshow(lr_astro.resize(hr_astro.size, Image.NEAREST)) # Nearest neighbor to emphasize pixels
axes[2].imshow(sr_astro_bicubic)
axes[3].imshow(sr_astro_dl)

# Formatting
for i, ax in enumerate(axes):
    ax.set_title(cols[i], fontsize=13, fontweight='bold' if i==3 else 'normal')
    ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# ==============================================================================
# Demo: Lightweight Inpainting (LaMa) - Simple Object
# Model: LaMa (Resolution-robust Large Mask Inpainting) ~200MB
# Data: Coffee Cup (skimage.data.coffee)
# Task: Remove the spoon from the image
# ==============================================================================

# 1. Install lightweight library
try:
    import simple_lama_inpainting
except ImportError:
    print("Installing LaMa...")
    !pip install -q simple_lama_inpainting

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from skimage import data
from simple_lama_inpainting import SimpleLama

# --- 1. Prepare Data (Simple Coffee Image) ---
def prepare_simple_data():
    """
    Loads coffee image and masks out the spoon.
    """
    # Load image (Standard sample: Cup of coffee)
    img_gt = Image.fromarray(data.coffee())

    # Create Mask
    w, h = img_gt.size
    mask_np = np.zeros((h, w), dtype=np.uint8)

    # Define a box mask over the spoon (Right side of the image)
    # Coordinates: [y1:y2, x1:x2]
    mask_np[150:280, 300:500] = 255

    mask = Image.fromarray(mask_np)

    # Create Observation (Black out the spoon area)
    img_masked_np = np.array(img_gt).copy()
    img_masked_np[mask_np == 255] = 0
    img_masked = Image.fromarray(img_masked_np)

    return img_gt, img_masked, mask

# --- 2. Inference ---
print("Initializing LaMa model...")
lama = SimpleLama() # Automatically downloads ~200MB model on first run

img_gt, img_masked, mask = prepare_simple_data()

print("Running Inpainting (Removing the spoon)...")
# Inference is fast on CPU or GPU
img_result = lama(img_masked, mask)

# --- 3. Visualization ---
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

axes[0].imshow(img_gt)
axes[0].set_title("1. Ground Truth (With Spoon)")

axes[1].imshow(img_masked)
axes[1].set_title("2. Input: Masked Image ($y$)")

axes[2].imshow(img_result)
axes[2].set_title("3. Output: LaMa Inpainting\n(Spoon Removed / Texture Restored)")

for ax in axes:
    ax.axis('off')

plt.tight_layout()
plt.show()

## 4. IP Summary: MAP Estimation vs. Uncertainty Quantification (UQ)

Bayesian Inversion is not just about finding the *best* image; it is about understanding **reliability**.

* **MAP Estimation (Point Estimate):**
    * **Goal:** Find the single most likely solution.
    * **Methodology:** Deterministic Deep Learning (like the **SwinIR** used above) excels here, producing visually sharp results.
    * **Limitation:** It lacks **Uncertainty Quantification**. We do not know if a recovered detail is real or a "hallucination" of the model.

* **Uncertainty Quantification (UQ):**
    * **Goal:** Characterize the full posterior distribution $p(x|y)$ to assess confidence.
    * **Methodology:** Classical mathematical methods (e.g., MCMC, Variational Inference) provide rigorous error bars and confidence intervals, which are often missing in standard deep learning.

**Application Focus:**
1.  **Medical Imaging & Geophysics (UQ focused):** reliability is paramount. We need to know the probability that a spot on an MRI is a tumor versus an artifact.
2.  **Computational Photography (MAP focused):** Visual quality is paramount. A plausible high-resolution texture is acceptable even if not statistically guaranteed.

## 5. Introduction to Data Assimilation

**Data Assimilation (DA)** fuses a mathematical model with noisy observations to optimally estimate the state of a system.

### Mathematical Formulation

**1. Stochastic Dynamics Model**
The evolution of the true state $v^\dagger$ is driven by dynamics $\Psi$ and process noise $\xi^\dagger$:
$$
\begin{align}
v_{j+1}^\dagger &= \Psi(v_j^\dagger) + \xi_j^\dagger, \quad j \in \mathbb{Z}^+,  \\
v_0^\dagger &\sim \mathcal{N}(m_0, C_0), \quad \xi_j^\dagger \sim \mathcal{N}(0, \Sigma) \text{ i.i.d.},
\end{align}
$$
* $v^\dagger$: The unknown True State.
* $\Psi$: The Dynamics Operator (Physics).
* $\Sigma$: Process Noise Covariance (Model uncertainty).

**2. Data Model**
We observe the system via an operator $h$ with measurement noise $\eta^\dagger$:
$$
\begin{align}
y_{j+1}^\dagger &= h(v_{j+1}^\dagger) + \eta_{j+1}^\dagger, \quad j \in \mathbb{Z}^+,  \\
\eta_{j+1}^\dagger &\sim \mathcal{N}(0, \Gamma) \text{ i.i.d.}
\end{align}
$$
* $y^\dagger$: The Observations.
* $h$: Observation Operator (maps state space to observation space).
* $\Gamma$: Observation Noise Covariance (Sensor uncertainty).

## 6: 2D Linear Gaussian Tracking

In this first experiment, we visualize the core mechanism of the **Kalman Filter** in a 2D plane. We can clearly see how the filter combines the model prediction (Prior) with the observation (Likelihood) to produce a refined estimate (Posterior).

### Experiment Setup
* **State**: $v = [x_1, x_2]^\top$.
* **Dynamics ($\Psi$)**: A rotation matrix $A$ that rotates the state by 0.35 radians at each step:
    \begin{bmatrix} \cos(0.35) & -\sin(0.35) \\ \sin(0.35) & \cos(0.35) \end{bmatrix}
* **Observation ($h$)**: We compare two scenarios:
    1.  **Full Observation**: We see the exact position ($x_1$ and $x_2$). $H = I_2$.
    2.  **Partial Observation**: We only see the first coordinate ($x_1$). $H = [1, 0]$.

### Visualization Key
* <font color="orange">**Orange (Prior)**</font>: Forecast before seeing data.
* <font color="red">**Red (Observation)**</font>: The noisy data.
    * **Dot**: Full observation ($x, y$).
    * **Vertical Line**: Partial observation (only $x$ is seen).
* <font color="blue">**Blue (Posterior)**</font>: The corrected estimate.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import ipywidgets as widgets
from IPython.display import display, clear_output

# Global Style
plt.style.use('seaborn-v0_8-whitegrid')

# --- 1. Helper Functions ---

OBS_CONFIGS = {
    'Full Observation (Identity)': {'matrix': np.eye(2), 'dim': 2},
    'Partial Observation (Only x)': {'matrix': np.array([[1.0, 0.0]]), 'dim': 1}
}

def get_cov_ellipse(cov, centre, n_std=2.448, **kwargs):
    """
    Returns a matplotlib Ellipse patch. n_std=2.448 is 95% CI in 2D.
    """
    cov = (cov + cov.T) / 2
    lambda_, v = np.linalg.eigh(cov)
    order = lambda_.argsort()[::-1]
    lambda_ = lambda_[order]
    v = v[:, order]
    angle = np.degrees(np.arctan2(v[1, 0], v[0, 0]))
    width, height = 2 * n_std * np.sqrt(np.maximum(lambda_, 0))
    return Ellipse(xy=centre, width=width, height=height, angle=angle, **kwargs)

def kalman_filter_step(m_prior, C_prior, y, H, Gamma):
    """
    Single Kalman Analysis Step.
    """
    S = H @ C_prior @ H.T + Gamma
    K = C_prior @ H.T @ np.linalg.inv(S)
    m_post = m_prior + K @ (y - H @ m_prior)
    C_post = (np.eye(len(m_prior)) - K @ H) @ C_prior
    return m_post, C_post

def generate_data(steps, obs_mode_key, dyn_noise_std, obs_noise_std):
    """
    Generates Truth (fixed start) and Noisy Observations.
    """
    theta = 0.35
    A = np.array([[np.cos(theta), -np.sin(theta)],
                  [np.sin(theta),  np.cos(theta)]])

    config = OBS_CONFIGS[obs_mode_key]
    H = config['matrix']
    obs_dim = config['dim']

    Sigma = np.eye(2) * (dyn_noise_std**2)
    Gamma = np.eye(obs_dim) * (obs_noise_std**2)

    np.random.seed(42)
    v_true = np.zeros((steps + 1, 2))
    v_true[0] = [5.0, 0.0] # Fixed Truth Start
    ys = []

    for k in range(steps):
        v_true[k+1] = A @ v_true[k] + np.random.multivariate_normal([0,0], Sigma)
        noise = np.random.multivariate_normal(np.zeros(obs_dim), Gamma)
        ys.append(H @ v_true[k+1] + noise)

    return v_true, ys, A, H, Sigma, Gamma

# --- 2. UI & Interaction Setup ---

# Obs Mode
w_obs_mode = widgets.Dropdown(
    options=list(OBS_CONFIGS.keys()),
    value='Full Observation (Identity)',
    description='Obs Mode:',
    layout=widgets.Layout(width='300px')
)

# Initial Guess Controls
w_guess_x = widgets.FloatText(value=5.0, description='Guess X:', step=0.5, layout=widgets.Layout(width='150px'))
w_guess_y = widgets.FloatText(value=0.0, description='Guess Y:', step=0.5, layout=widgets.Layout(width='150px'))

# Added style to ensure label is seen
w_init_std = widgets.FloatSlider(
    value=1.0, min=0.1, max=3.0, step=0.1,
    description='Guess Sigma:',
    continuous_update=False,
    style={'description_width': 'initial'}
)

# Noise Controls
w_dyn_noise = widgets.FloatSlider(value=0.01, min=0.001, max=0.05, step=0.001, description='Dyn Noise:', continuous_update=False)
w_obs_noise = widgets.FloatSlider(value=1.0, min=0.1, max=3.0, step=0.1, description='Obs Noise:', continuous_update=False)

# Button: Reduced width
btn_run = widgets.Button(
    description='Run Kalman Filter',
    button_style='success',
    icon='play',
    layout=widgets.Layout(width='200px')
)

out_plot = widgets.Output()

def on_button_clicked(b):
    btn_run.disabled = True
    btn_run.description = "Running..."

    with out_plot:
        clear_output(wait=True)
        try:
            # 1. Retrieve Parameters
            obs_mode = w_obs_mode.value
            dyn_std = w_dyn_noise.value
            obs_std = w_obs_noise.value

            guess_mean = np.array([w_guess_x.value, w_guess_y.value])
            init_std = w_init_std.value
            steps = 12

            # 2. Compute
            v_true, ys, A, H, Sigma, Gamma = generate_data(steps, obs_mode, dyn_std, obs_std)

            m_curr = guess_mean
            C_curr = np.eye(2) * (init_std**2)

            priors = []
            posts = [(m_curr, C_curr)]

            for k in range(steps):
                # Forecast
                m_prior = A @ m_curr
                C_prior = A @ C_curr @ A.T + Sigma
                priors.append((m_prior, C_prior))

                # Analysis
                m_post, C_post = kalman_filter_step(m_prior, C_prior, ys[k], H, Gamma)
                posts.append((m_post, C_post))
                m_curr, C_curr = m_post, C_post

            # 3. Visualization
            fig, ax = plt.subplots(figsize=(10, 8))

            # Plot Truth
            ax.plot(v_true[:, 0], v_true[:, 1], 'k-', alpha=0.4, label='Truth (Fixed Start)', linewidth=2)
            ax.scatter(v_true[:, 0], v_true[:, 1], c='k', s=20)

            # Highlight Initial Guess
            ell_init = get_cov_ellipse(posts[0][1], posts[0][0], edgecolor='green', linestyle='-', facecolor='green', alpha=0.1)
            ax.add_patch(ell_init)
            ax.plot(posts[0][0][0], posts[0][0][1], 's', color='green', label='Initial Guess (t=0)')

            # Loop steps
            for k in range(steps):
                m_pri, C_pri = priors[k]
                m_pos, C_pos = posts[k+1]
                y_val = ys[k]
                true_pos = v_true[k+1]

                # --- Plot Obs ---
                if obs_mode == 'Full Observation (Identity)':
                    ax.plot(y_val[0], y_val[1], 'r*', markersize=10, alpha=0.7)
                    # Link line
                    ax.plot([true_pos[0], y_val[0]], [true_pos[1], y_val[1]],
                            color='red', linestyle='--', alpha=0.3, linewidth=1)
                else:
                    # Solid red line for Partial Obs
                    ax.axvline(x=y_val[0], color='red', linestyle='-', alpha=0.3, linewidth=1.5)
                    # Link line (horizontal)
                    ax.plot([true_pos[0], y_val[0]], [true_pos[1], true_pos[1]],
                            color='red', linestyle='--', alpha=0.3, linewidth=1)

                # --- Prior (Orange) ---
                ell_pri = get_cov_ellipse(C_pri, m_pri, edgecolor='orange', linestyle='--', facecolor='none', alpha=0.6)
                ax.add_patch(ell_pri)
                ax.plot(m_pri[0], m_pri[1], 'x', color='orange', markersize=6)

                # --- Posterior (Blue) ---
                ell_pos = get_cov_ellipse(C_pos, m_pos, edgecolor='blue', linestyle='-', facecolor='blue', alpha=0.1)
                ax.add_patch(ell_pos)
                ax.plot(m_pos[0], m_pos[1], '.', color='blue', markersize=6)

                # --- Arrow ---
                ax.annotate("", xy=(m_pos[0], m_pos[1]), xytext=(m_pri[0], m_pri[1]),
                            arrowprops=dict(arrowstyle="->", color="gray", alpha=0.5))

            # Legend
            ax.plot([], [], 'x', color='orange', label='Prior (Forecast)')
            ax.plot([], [], 'r*', label='Observation')
            if obs_mode != 'Full Observation (Identity)':
                ax.plot([], [], 'r-', label='Observed Subspace', alpha=0.5)
            ax.plot([], [], 'o', color='blue', label='Posterior (Analysis)')

            ax.set_xlim(-10, 10)
            ax.set_ylim(-10, 10)
            ax.set_aspect('equal')
            ax.set_title(f'Kalman Filter Tracking ({obs_mode})', fontsize=14)
            ax.set_xlabel('$x_1$')
            ax.set_ylabel('$x_2$')
            ax.legend(loc='upper left', frameon=True)
            ax.grid(True, linestyle='--', alpha=0.6)

            display(fig)
            plt.close(fig)

        except Exception as e:
            print(f"Error: {e}")
        finally:
            btn_run.disabled = False
            btn_run.description = 'Run Kalman Filter'

btn_run.on_click(on_button_clicked)

# Layout
ui_guess = widgets.HBox([w_guess_x, w_guess_y, w_init_std])
ui_noise = widgets.HBox([w_dyn_noise, w_obs_noise])
ui = widgets.VBox([
    widgets.HTML("<b>1. Settings:</b>"),
    w_obs_mode,
    widgets.HTML("<b>2. Initial Guess (Prior at t=0):</b>"),
    ui_guess,
    widgets.HTML("<b>3. System Noise:</b>"),
    ui_noise,
    widgets.HTML("<br>"),
    btn_run
])

print("Configure and Run:")
display(ui, out_plot)
on_button_clicked(None)

## 7. Particle Filter on Lorenz '63 (Partial Observation)

In this experiment, we explore the Particle Filter for the Lorenz '63 system. Unlike the Kalman Filter, which assumes all errors are Gaussian, the Particle Filter represents the probability distribution using a swarm of discrete particles.

We specifically use the **Bootstrap Particle Filter**, which is a standard implementation of the sequential importance resampling method.

### 1. Dynamics Model (Lorenz-63)
The system is governed by the chaotic Lorenz equations ($\sigma=10, \rho=28, \beta=8/3$):
$$
\begin{align}
\frac{dx}{dt} &= 10(y-x) \\
\frac{dy}{dt} &= x(28-z)-y \\
\frac{dz}{dt} &= xy - \frac{8}{3} z
\end{align}
$$
We simulate this using a robust Runge-Kutta 4 (RK4) solver.

### 2. Observation Model
Crucially, this is a **partial observation** experiment. We assume we can only measure the first variable, $x$.
The filter must infer the hidden variables $y$ and $z$ through the physical correlations maintained by the particle swarm.

### 3. Algorithm Steps (Bootstrap PF)
1.  **Forecast**: Each particle evolves independently according to the physics model plus random process noise.
2.  **Weighting**: Particles are weighted based on the likelihood $p(y|x)$.
3.  **Resampling**: High-weight particles are duplicated, while low-weight particles are discarded.

### 4. Parameters & Interaction
* **Assimilation Window**: $\Delta t_{obs} = 0.15$.
* **Solver Step**: $dt = 0.03$ (5 substeps per window).
* **Noise**: System noise $\Sigma=0.01$, Observation noise $\Gamma=1.0$.

Two papers related to the number of particles of the particle filter, and the mode collapse issue when the number of particles is not enough:

1. Curse-of-dimensionality revisited: Collapse of the particle filter in very large scale systems
https://arxiv.org/abs/0805.3034

2. Obstacles to High-Dimensional Particle Filtering
https://journals.ametsoc.org/view/journals/mwre/136/12/2008mwr2529.1.xml

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import ipywidgets as widgets
from IPython.display import display, clear_output

# Global Style
plt.style.use('seaborn-v0_8-whitegrid')

# --- 1. Physics & Solver ---

def lorenz_dynamics(v):
    """
    Description: Lorenz-63 derivatives.
    Input: v=[x, y, z]
    Output: dv/dt
    """
    sigma, rho, beta = 10.0, 28.0, 8.0/3.0
    x, y, z = v
    dxdt = sigma * (y - x)
    dydt = x * (rho - z) - y
    dzdt = x * y - beta * z
    return np.array([dxdt, dydt, dzdt])

def rk4_integrate(v, dt, n_steps):
    """
    Description: Propagates state v forward using RK4.
    """
    curr_v = v.copy()
    for _ in range(n_steps):
        k1 = lorenz_dynamics(curr_v)
        k2 = lorenz_dynamics(curr_v + 0.5 * dt * k1)
        k3 = lorenz_dynamics(curr_v + 0.5 * dt * k2)
        k4 = lorenz_dynamics(curr_v + dt * k3)
        curr_v = curr_v + (dt / 6.0) * (k1 + 2*k2 + 2*k3 + k4)
    return curr_v

# --- 2. Particle Filter Logic ---

def particle_filter_run(n_particles, steps, y_obs, obs_sigma, dyn_sigma, v0_fixed):
    """
    Description: Runs SIR Particle Filter with Partial Observation (X only).
    Input: y_obs is expected to be 1D array (sequence of x values).
    """
    # Initialize particles around a guessed state
    v0_guess = v0_fixed + np.random.normal(0, 3.0, 3)
    particles = v0_guess + np.random.normal(0, 3.0, (n_particles, 3))

    weights = np.ones(n_particles) / n_particles

    # History
    particles_hist = [particles.copy()]
    mean_hist = [np.mean(particles, axis=0)]

    # Params
    dt_rk = 0.03
    T_assim = 0.15
    n_substeps = int(T_assim / dt_rk)

    for k in range(steps):
        # A. Forecast
        for i in range(n_particles):
            pred = rk4_integrate(particles[i], dt_rk, n_substeps)
            noise = np.random.normal(0, dyn_sigma, 3)
            particles[i] = pred + noise

        # B. Update (Likelihood based on X ONLY)
        # y_obs[k] is the observed scalar X
        y_scalar = y_obs[k]

        # Distance squared on X dimension only (index 0)
        d2 = (particles[:, 0] - y_scalar)**2

        # Log-weights
        log_weights = -0.5 * d2 / (obs_sigma**2)

        # Softmax normalization
        max_log = np.max(log_weights)
        weights = np.exp(log_weights - max_log)
        weight_sum = np.sum(weights)
        if weight_sum > 0:
            weights /= weight_sum
        else:
            weights = np.ones(n_particles)/n_particles

        # C. Resampling
        indices = np.random.choice(n_particles, size=n_particles, p=weights)
        particles = particles[indices]

        # Store
        particles_hist.append(particles.copy())
        mean_hist.append(np.mean(particles, axis=0))

    return particles_hist, np.array(mean_hist)

def generate_lorenz_truth(steps, v0, dyn_sigma, obs_sigma):
    """
    Description: Generates Truth (3D) and Observations (1D, X only).
    """
    dt_rk = 0.03
    T_assim = 0.15
    n_substeps = int(T_assim / dt_rk)

    v_true = np.zeros((steps + 1, 3))
    v_true[0] = v0
    ys = []

    curr = v0.copy()
    for k in range(steps):
        curr = rk4_integrate(curr, dt_rk, n_substeps)
        curr += np.random.normal(0, dyn_sigma, 3)
        v_true[k+1] = curr

        # Generate Observation (X only)
        y_x = curr[0] + np.random.normal(0, obs_sigma)
        ys.append(y_x)

    return v_true, np.array(ys)

# --- 3. UI & Visualization ---

w_particles = widgets.IntSlider(
    value=100, min=10, max=1000, step=10,
    description='Particles:',
    continuous_update=False,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

btn_pf_run = widgets.Button(
    description='Run Particle Filter',
    button_style='primary',
    icon='play'
)

out_pf = widgets.Output()

def on_pf_click(b):
    btn_pf_run.disabled = True
    btn_pf_run.description = "Computing..."
    with out_pf:
        clear_output(wait=True)
        try:
            # Fixed Seed for Reproducibility
            np.random.seed(42)

            # 1. Setup
            steps = 25
            v0_fixed = np.array([-5.0, -5.0, 20.0])
            dyn_sigma = 0.01
            obs_sigma = 1.0 # fixed
            n_part = w_particles.value

            # 2. Run
            v_true, ys_obs = generate_lorenz_truth(steps, v0_fixed, dyn_sigma, obs_sigma)
            part_hist, mean_hist = particle_filter_run(n_part, steps, ys_obs, obs_sigma, dyn_sigma, v0_fixed)

            # 3. Plot (1 row, 3 columns)
            fig = plt.figure(figsize=(18, 5))
            time_points = np.arange(steps + 1)

            # --- Plot 1: X Dimension (Observed) ---
            ax1 = fig.add_subplot(1, 3, 1)
            ax1.plot(time_points, v_true[:, 0], 'k-', lw=2, label='Truth X')
            ax1.plot(time_points[1:], ys_obs, 'r*', ms=10, label='Obs X') # Show Obs Here
            ax1.plot(time_points, mean_hist[:, 0], 'b--', lw=2, label='PF Est X')

            # Particles Background
            display_N = min(n_part, 50)
            for t in range(steps + 1):
                subset = part_hist[t][:display_N]
                ax1.scatter([t]*len(subset), subset[:, 0], c='blue', alpha=0.1, s=5)

            ax1.set_title('Observed State: X')
            ax1.set_xlabel('Step')
            ax1.set_ylabel('X Value')
            ax1.legend()

            # --- Plot 2: Y Dimension (Unobserved) ---
            ax2 = fig.add_subplot(1, 3, 2)
            ax2.plot(time_points, v_true[:, 1], 'k-', lw=2, label='Truth Y')
            # NO Obs plot here
            ax2.plot(time_points, mean_hist[:, 1], 'b--', lw=2, label='PF Est Y')

            # Particles Background
            for t in range(steps + 1):
                subset = part_hist[t][:display_N]
                ax2.scatter([t]*len(subset), subset[:, 1], c='blue', alpha=0.1, s=5)

            ax2.set_title('Hidden State: Y (Inferred)')
            ax2.set_xlabel('Step')
            ax2.set_ylabel('Y Value')
            ax2.legend()

            # --- Plot 3: 3D Trajectory ---
            ax3 = fig.add_subplot(1, 3, 3, projection='3d')
            ax3.plot(v_true[:,0], v_true[:,1], v_true[:,2], 'k-', lw=1.5, label='Truth', alpha=0.6)
            ax3.scatter(v_true[0,0], v_true[0,1], v_true[0,2], c='k', marker='o', s=30, label='Start')

            # Particles (Step by Step) - NO Obs Red Stars
            for t in range(1, steps + 1):
                p_curr = part_hist[t]
                if n_part > 300:
                    idx = np.random.choice(n_part, 300, replace=False)
                    p_curr = p_curr[idx]
                ax3.scatter(p_curr[:,0], p_curr[:,1], p_curr[:,2], c='blue', s=2, alpha=0.1, depthshade=True)

            ax3.set_title('3D Particle Cloud')
            ax3.set_xlabel('X')
            ax3.set_ylabel('Y')
            ax3.set_zlabel('Z')
            ax3.view_init(elev=20, azim=-60)

            plt.tight_layout()
            display(fig)
            plt.close(fig)

        except Exception as e:
            print(f"Error: {e}")
        finally:
            btn_pf_run.disabled = False
            btn_pf_run.description = 'Run Particle Filter'

btn_pf_run.on_click(on_pf_click)

# Layout
ui_pf = widgets.VBox([
    widgets.HTML("<b>Adjust Particle Count (Observe stability at low N vs high N):</b>"),
    widgets.HBox([w_particles, btn_pf_run])
])

print("Particle Filter Demo (Partial Obs: X only):")
display(ui_pf, out_pf)
on_pf_click(None)

## 8. Real-World Impact: Why Weather Forecasting Needs DA?

### Core Concept: Refining vs. Making Prediction
It is crucial to understand that **Data Assimilation (DA) does not "make" predictions**—that is the job of the physical model ($\Psi$).
* **The Model's Job**: To calculate the future state based on physics.
* **DA's Job**: To **refine** that calculation by constantly correcting the trajectory with new data.

### The Challenge: The "Butterfly Effect"
In **Numerical Weather Prediction (NWP)**, we use massive supercomputers to forward the chaotic dynamics:
1.  **Sensitivity to Initial Conditions**: A tiny error in measuring today's temperature (even $0.01^\circ$C) will grow exponentially. Without correction, even a perfect model would become useless after ~14 days.
2.  **Model Imperfections**: No model is perfect. Approximations in physics (e.g., how clouds form) introduce systematic drift.

### The Solution: DA as the "Reality Check"
Data Assimilation acts as the necessary **correction mechanism**:
* Every 6 hours, millions of observations (satellites, balloons, aircraft) are collected globally.
* The model state is not simply "overwritten" by these measurements. Instead, noisy data is statistically fused with the model's previous forecast (the Prior) using DA methods (like **4D-Var** or **EnKF**).
* **Result**: An optimal **"Analysis"** is generated—serving as the best possible starting point for the *next* prediction.

Without this **refinement step**, modern weather forecasting would be impossible. The revolution of forecast accuracy over the last 30 years is largely attributed to better DA algorithms, not just faster computers.